# Kaggle House Prices - Clean Training Data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font_scale=1.3, style="ticks")
fs = (14, 9) # make the figure wider than the default

## Load the Data

* Load the test set and take a look

In [2]:
# Load the test data
houses = pd.read_csv("data/train.csv")

# Take a look
houses.shape

(1460, 81)

In [ ]:
# Sample
houses.sample(10)

## Inspect and correct data types

In [ ]:
# Column names and data types
houses.dtypes[0:40]

In [ ]:
# Column names and data types
houses.dtypes[40:80]

In [ ]:
houses.dtypes[80:]

In [3]:
# Replace column names that start with an invalid character
col_names = [key for key in dict(houses.dtypes) if re.search("\.",key) != None]
print("periods: 0") if len(col_names) == 0 else print("periods:",col_names)
col_names = [key for key in dict(houses.dtypes) if re.search("__",key) != None]
print("multiple underscores: 0") if len(col_names) == 0 else print("multiple underscores:",col_names)
col_names = [key for key in dict(houses.dtypes) if re.search("^[^A-Z]",key) != None]
print("invalid start char: 0") if len(col_names) == 0 else print("invalid start char:",col_names)

periods: 0
multiple underscores: 0
invalid start char: ['1stFlrSF', '2ndFlrSF', '3SsnPorch']


### Corrective Actions

* Rename: '1stFlrSF', '2ndFlrSF', '3SsnPorch' to: 'FirstFlrSF', 'SecondFlrSF', 'ThirdSsnPorch'

In [4]:
# Rename columns starting with numbers
new_col_names = ['FirstFlrSF', 'SecondFlrSF', 'ThirdSsnPorch']
new_col_names = dict(zip(col_names,new_col_names))
houses.rename(columns=new_col_names, inplace=True)

## Inspect Missing Data

In [8]:
# What % data (if any) is missing for each column?
nulls = houses.isnull().sum()
nulls = pd.DataFrame(nulls)
nulls.rename(columns={0:"cnt_missing"},inplace=True)
nulls = nulls[nulls.cnt_missing>0]
nulls['pct_missing'] = round(nulls.cnt_missing/houses.shape[0] * 100,2)
nulls.sort_values(by="pct_missing",ascending=False)

,cnt_missing,pct_missing
PoolQC,1453,99.52
MiscFeature,1406,96.30
Alley,1369,93.77
Fence,1179,80.75
FireplaceQu,690,47.26
LotFrontage,259,17.74
GarageType,81,5.55
GarageYrBlt,81,5.55
GarageFinish,81,5.55
GarageQual,81,5.55


### Corrective Actions

* PoolQC, MiscFeature, Alley, and Fence are missing for > 80% of rows.  Discard these columns.
* FireplaceQu and LotFrontage are missing for > 15% of rows, can we impute these?

In [ ]:
# Drop columns PoolQC, MiscFeature, Alley, and Fence
houses.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence'], axis=1, inplace=True)
houses.shape

In [ ]:
# Inspect some values for FireplaceQu and LotFrontage

# Possible values for FireplaceQu
# Ex   Excellent - Exceptional Masonry Fireplace
# Gd   Good - Masonry Fireplace in main level
# TA   Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
# Fa   Fair - Prefabricated Fireplace in basement
# Po   Poor - Ben Franklin Stove
# NA   No Fireplace
# sns.catplot(x="FireplaceQu", kind="count", palette="ch:.25", data=houses)

In [9]:
# Fireplace quality is missing where Fireplaces == 0
houses.loc[houses.Fireplaces==0].shape[0]

690

In [11]:
# Perhaps this means we can roll the two together into one variable?
houses.Fireplaces.unique()

array([0, 1, 2, 3])

In [ ]:
# x_axis = sorted(houses.LotFrontage.unique())
# x_axis = [x for x in x_axis if str(x) != 'nan']
# lotf = sns.catplot(x="LotFrontage", kind="count", palette="ch:.25", data=houses, height=9, aspect=3)
# lotf.set_xticklabels(rotation=90, labels = x_axis)

In [12]:
houses.loc[houses.LotFrontage.isnull()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
12,13,20,RL,NaN,12968,Pave,NaN,IR2,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,144000
14,15,20,RL,NaN,10920,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,GdWo,NaN,0,5,2008,WD,Normal,157000
16,17,20,RL,NaN,11241,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,700,3,2010,WD,Normal,149000
24,25,20,RL,NaN,8246,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,5,2010,WD,Normal,154000
31,32,20,RL,NaN,8544,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2008,WD,Normal,149350
42,43,85,RL,NaN,9180,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,12,2007,WD,Normal,144000
43,44,20,RL,NaN,9200,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,7,2008,WD,Normal,130250
50,51,60,RL,NaN,13869,Pave,NaN,IR2,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2007,WD,Normal,177000
64,65,60,RL,NaN,9375,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,NaN,0,2,2009,WD,Normal,219500


## Remove any other unwanted columns

* ID doesn't make sense as a predictor

In [ ]:
houses.drop(["Id"], axis = 1, inplace=True)
houses.shape

## Save Output

* Save the cleaned data to a new file

In [ ]:
houses.to_csv('data/cleaned_data/cleaned_train.csv')